# Test 1

Not working, cannot save results

In [11]:
import digitalhub as dh

p_name = "cdt-parallelization"
project = dh.get_or_create_project(p_name)

In [47]:
func = project.new_function(
    name="compute_scenario_worker",
    kind="python",
    python_version="PYTHON3_12",
    code_src="civic_digital_twins/dt_model/simulation/utils.py",
    handler="compute_scenario_worker",
)

In [48]:
run = func.run(
    "job",
    inputs={
        "scenario_config": {},
        "parallel_execution": True,
        "early_stopping": False,
    },
    wait=True,
)

2025-11-14 09:49:06,456 - INFO - Waiting for run 81f9d8b4e93a4991817e17c17debc58c to finish...
2025-11-14 09:49:11,473 - INFO - Waiting for run 81f9d8b4e93a4991817e17c17debc58c to finish...
2025-11-14 09:49:16,489 - INFO - Waiting for run 81f9d8b4e93a4991817e17c17debc58c to finish...
2025-11-14 09:49:21,507 - INFO - Run 81f9d8b4e93a4991817e17c17debc58c finished in 15.05 seconds.


In [41]:
scenarios = {
    "Base": {},
    "GoodWeather": {"CV_weather": ["good", "unsettled"]},
    "BadWeather": {"CV_weather": ["bad"]},
}

for name, config in scenarios.items():
    run = func.run(
        "job",
        inputs={
            "scenario_config": config,
            "parallel_execution": True,
            "early_stopping": False,
        },
        wait=False,
    )

BadRequestError: Bad request. Response: {"status":400,"message":"Cannot deserialize value of type `java.lang.String` from Object value (token `JsonToken.START_OBJECT`)\n at [Source: UNKNOWN; byte offset: #UNKNOWN] (through reference chain: it.smartcommunitylabdhub.runtime.python.specs.PythonRunSpec[\"inputs\"]->java.util.LinkedHashMap[\"scenario_config\"])","errorCode":null}.

# TEST2

In [49]:
%%writefile "parallel_run.py"


import digitalhub as dh
from civic_digital_twins.dt_model.simulation.utils import compute_scenario_worker
import pickle

@handler(outputs=["result"])
def main(project, di):
    # download as local file
    scenario_name, result = compute_scenario_worker({}, True, False)
    outfile = f"{scenario_name}.pkl"
    
    with open(outfile, "wb") as f:
        pickle.dump(result, f)

    project.log_artifact(kind="artifact", name=outfile, source=outputfile)
        
    # persist result
    return result

Overwriting parallel_run.py


In [50]:
func = project.new_function(name="run",
                            kind="python",
                            python_version="PYTHON3_12",
                            code_src="parallel_run.py",
                            handler="main")

In [51]:
run = func.run("job",
               wait=True)

2025-11-14 09:51:30,169 - INFO - Waiting for run c143813fda9a47f0b3633478c3a910bf to finish...
2025-11-14 09:51:35,185 - INFO - Waiting for run c143813fda9a47f0b3633478c3a910bf to finish...
2025-11-14 09:51:40,205 - INFO - Waiting for run c143813fda9a47f0b3633478c3a910bf to finish...
2025-11-14 09:51:45,224 - INFO - Waiting for run c143813fda9a47f0b3633478c3a910bf to finish...
2025-11-14 09:51:50,242 - INFO - Run c143813fda9a47f0b3633478c3a910bf finished in 20.07 seconds.
